In [1]:
import os
import sys
import pandas as pd
import time
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# NAVER BLOG Crawling
- title/url 

In [71]:
# 검색어 입력
query = input("검색어 : ")


# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

print(startDate, endDate)


# Chrome driver 환경설정 및 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)
base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
driver.get(base_url)
time.sleep(1)


# 총 검색 결과
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
search_number = soup.find(class_="search_number")
print("총 블로그글 개수 : ", search_number.text)


# 연관 검색어 
sub_keywords_data = soup.select('div.area_keyword > div.list')
sub_keywords = []
for i in sub_keywords_data: 
    sub_keywords.append(i.get_text())

print("연관검색어:",*sub_keywords)


# blog url/title 가져오기
total_num = search_number.text.replace("건",'').replace(",",'')
total_num = int(total_num)
page_num = 1
end_page=total_num//7+1
end_page
contents_num = 7

blog_title_lst = []
blog_url_lst = []

while contents_num == 7 : 
    search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
    driver.get(search_url)
    time.sleep(0.5)
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
    
    # 블로그 제목, url 가져오기
    posts_data = soup.select('div.info_post > div.desc > a.desc_inner')

    for post in posts_data :
        title=post.get_text().replace('\n','').strip()
        href=post.attrs['href']
        blog_title_lst.append(title)
        blog_url_lst.append(href)
    
    page_num +=1
    contents_num = len(posts_data)

print(blog_title_lst)
print(blog_url_lst)

2024-05-03 2024-05-10
총 블로그글 개수 :  335건
연관검색어: 
회기역 맛집회기역 경의중앙선회기역 1호선회기역 경의중앙선 시간표회기역 카페회기역 시간표회기역 술집회기역 미용실회기역 경춘선회기역 1호선 시간표회기역 미래사주회기역 오피스텔회기역 파전골목회기역 놀거리회기역 파전회기역 아파트회기역 꽃집회기역 대학교청량리역상봉역


['회기왕갈비탕, 혼밥 하기 좋은 회기역 맛집', '또 갈수밖에 없는 회기역 술집 와레와레 또 갔다왔지요.', '회기역 맛집 또와 양꼬치에서 무한리필 즐기기 회기 양꼬치', '회기역 조용한 카페 / 비건디저트 벨리스 후기', "회기역 삼겹살 맛집 '고깃집도훈' 냉삼 1인분에 9,000원 가성비 최고!", '1년 만에 재방문한 회기역 홍익돈까스 내돈내산 찐 후기', '회기역 점심맛집 외대 맛집 추천 회기왕갈비탕', '회기역 외대 맛집 회기왕갈비탕', '[회기] 동대문구 회기역 맛집 신쭈 회기점 - 회기역쭈꾸미와 철판 쭈꾸미삼겹살 추천', '휘경동 회기역 근처 맛집 고기가 푸짐한 회기왕갈비탕', '회기역 술집 네 가지 소스 독특한 양고기집 양마담', '경희대 데이트 팔각도 경희대점 회기역 회식', '회기역 스캔 추천 [스캔데이] 시립대 경희대 책스캔', '회기역 맛집 [인생 소곱창] 인생맛집 회기역 데이트맛집', "비올때 생각나는 회기역 파전골목의 대표맛집 '이모네 왕파전'", '회기역 맛집 데이트 매스 플레이트 파스타 목살스테이크 샐러드', '[회기역맛집 | 경희대맛집 | 회기역쭈꾸미] 파전골목만 유명한게 아닙니다. 엄청난...', '경희대 회기역 맛집 내돈내산 규범 곱창 찐 맛집 인정~!', '새로 생긴 회기역 디저트카페 "벨리스" / 비건디저트', '회기역 개인회생 개인파산 신용회복, 법무사 빚탈출을 꿈꾸는 분들을 위해', '휘경동 맛집 회기역 밥집 회기왕갈비탕', '교대근무 직장인 5월 대회 준비 / 회기pt, 회기헬스, 회기역pt 애프터짐', '회기역에서 놀았어요. 인기있는 맛집 코타와 카페 오후 한시 십일분', 

In [72]:
# 확인
print(len(blog_title_lst),len(blog_url_lst))

335 335


- blog 본문 가져오기(test)

In [ ]:
# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)


# 해당 blog 불러오기 
url = blog_url_lst[0]
driver.get(url)
time.sleep(1)


# iframe 접근 
driver.switch_to.frame('mainFrame')


# 제목 가져오기
overlays_title = ".se-module.se-module-text.se-title-text"
html_title = driver.find_element(By.CSS_SELECTOR, overlays_title)
title = html_title.text
print(title)


# 포스팅 날짜 가져오기
overlays_publishDate = ".se_publishDate.pcol2"
html_publishDate = driver.find_element(By.CSS_SELECTOR, overlays_publishDate)
date = html_publishDate.text
print(date)

# 본문 가져오기 
overlays_content = ".se-component.se-text.se-l-default"
html_content = driver.find_elements(By.CSS_SELECTOR, overlays_content)
contents_lst = []
for content in html_content : 
    contents_lst.append(content.text.replace('\n',''))
    

print(contents_lst)

# 해시태그 가져오기 

# title = html_title.text 
# title





- blog 본문 크롤링

In [74]:
# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)

crawling_blog_data = []


# blog 본문 가져오기 
for i in range(10) :    
    url = str(blog_url_lst[i])
    driver.get(url)
    time.sleep(1)
    
    
    # 본문 크롤링 예외 처리 
    try :
        # iframe 접근 
        driver.switch_to.frame('mainFrame')

        
        # 제목 가져오기
        overlays_title = ".se-module.se-module-text.se-title-text"
        html_title = driver.find_element(By.CSS_SELECTOR, overlays_title)
        title = html_title.text

        
        # 작성 날짜 가져오기
        overlays_publishDate = ".se_publishDate.pcol2"
        html_publishDate = driver.find_element(By.CSS_SELECTOR, overlays_publishDate)
        date = html_publishDate.text

        
        # 본문 가져오기
        overlays_content = ".se-component.se-text.se-l-default"
        html_content = driver.find_elements(By.CSS_SELECTOR, overlays_content)
        contents_lst = []
        for content in html_content : 
            contents_lst.append(content.text.replace('\n',''))
        content_str = ''.join(contents_lst)

        
        # 크롤링한 내용 담기 
        crawling_blog_data.append({"title":title, "date":date, "content":content_str})
        
        # driver 종료 
        driver.close
        time.sleep(1)
        
        
    # 에러 발생시 다음 블로그로 이동
    except : 
        print("error",i,title)
        driver.close()
        time.sleep(1)

driver.close()

In [77]:
print(crawling_blog_data)

[{'title': '회기왕갈비탕, 혼밥 하기 좋은 회기역 맛집', 'date': '2024. 5. 8. 8:46', 'content': '안녕하세요이나입니다:)오늘은 혼밥 하기 좋은 회기역 맛집인 회기왕갈비탕에 방문한 후기를 들고 왔어요회기역 근처에서 촬영을 하고 혼밥을 해야 해서 뭘 먹지 고민하다가 혼밥 하기 좋다는 이야길 듣고 바로 후다닥 달려갔습니다 ㅎㅎ회기역에서 도보 5분 정도 거리로 굉장히 접근성도 좋아요ㅎㅎ그리고 외대 경희대 학생들도 가까워서 방문하기 좋을 것 같더라고요회기역 맛집은 석박지를 무려 직접 만든다고 해요힘드시겠다.. 그렇지만 소비자는 너무 맛있어요 츄릅김치 국내산? 보장된 거잖아요 히히회기왕갈비탕은 외부에서 봤을 때도 넓어 보였는데 들어가서 봐도 진짜 넓어요 굿굿쾌적하고 좌석도 많아서 단체로 와서 먹기에도 괜찮겠더라고요무려 1인 테이블도 있다는 점!! 혼밥 하기 좋은 회기역 맛집답게 1인 테이블까지~!~!근데 저는 혼밥 만렙이라서 4인 테이블 앉았어요점심시간 지나고 가서 자리가 많았기 때문이죠!!소갈비찜, 갈비탕 아롱사태 갈비탕 이렇게 메뉴가 깔끔하게 구성되어 있어요원래 맛집은 메뉴가 많이 없는 거 아시죠???기본 반찬 구성은 석박지, 쌈장, 고추, 마늘 이렇게 제공돼요!!기본 반찬 외에 필요한 건 셀프 코너에서!!셀프 코너에는 배추김치도 있어요근데 지금 포스팅하면서 알았는데 보리강정 먹어도 되는 거였냐고요진짜 몰랐네 알았으면 먹었는데…이거 진짜 좋아하는데속상하다…다음에는 꼭 먹어주겠다 보리강정 ㅡㅡ보리강정을 1봉지 3,000원2봉지 5,000원에 판매하고갈비탕 육수까지 같이 판매하고 있어요진짜 회기왕갈비탕 먹어보면 육수 따로 살만해요진짜 맛있거든요 ㅎㅎ혼밥 하기 좋은 회기역 맛집인 회기왕갈비탕은 무려 밥도 무한리필!!제가 아는 지인은 국밥 한 그릇에 세 공기, 다섯 공기 이렇게 먹는데 추천해 주고 싶네요 ㅋㅋㅋㅋㅋㅋ미숫가루도 후식으로 먹을 수 있게 준비되어 있어요열었는데 알아서 막 움직여서 놀랐어요 ㅋㅋㅋㅋㅋㅋㅋ진짜 당황 ㅇㅅㅇ혼밥 

In [82]:
df=pd.DataFrame(crawling_blog_data)
df

,title,date,content
0,"회기왕갈비탕, 혼밥 하기 좋은 회기역 맛집",2024. 5. 8. 8:46,안녕하세요이나입니다:)오늘은 혼밥 하기 좋은 회기역 맛집인 회기왕갈비탕에 방문한 후...
1,또 갈수밖에 없는 회기역 술집 와레와레 또 갔다왔지요.,2024. 5. 6. 16:34,안녕하세요 별숌이에요~예전에 한 번 다녀온적이 있는회기역 술집 '와레와레'지난번 안...
2,회기역 맛집 또와 양꼬치에서 무한리필 즐기기 회기 양꼬치,2024. 5. 6. 16:38,안녕하세요~ 미친 또라이 미또입니다최근에 양꼬치가 너무 먹고싶어서한동안 양고기만 먹...
3,회기역 조용한 카페 / 비건디저트 벨리스 후기,2024. 5. 4. 11:02,오늘은 회기역 비건카페로 유형한벨리스 다녀와봤어요 :)영업시간 10:00-20:00...
4,"회기역 삼겹살 맛집 '고깃집도훈' 냉삼 1인분에 9,000원 가성비 최고!",2024. 5. 7. 5:58,안녕하세요.🥰이번 주 주말에도 예랑이와 신혼집에 들어갈침대 계약을 하고 저녁으로 고...
5,1년 만에 재방문한 회기역 홍익돈까스 내돈내산 찐 후기,2024. 5. 7. 12:29,안녕하세요 여러분~!!!오랜만의 포스팅이죠!!5월 4일부터 어제까지본가에 다녀오느라...
6,회기역 점심맛집 외대 맛집 추천 회기왕갈비탕,2024. 5. 7. 19:10,안녕하세요 뚜둡이에요퇴근하고 회기역에 제본하러 들렀다가몸보신 겸 뜨끈한 밥 한 그릇...
7,회기역 외대 맛집 회기왕갈비탕,2024. 5. 4. 11:54,"회기역 맛집, 외대 맛집, 갈비탕 맛집 찾으실 때회기왕갈비탕 방문해 보시길 추천드립..."
8,[회기] 동대문구 회기역 맛집 신쭈 회기점 - 회기역쭈꾸미와 철판 쭈꾸미삼겹살 추천,12시간 전,매콤한 음식이 땡겨 신쭈 회기점에 방문했습니다!!뭔가 쭈꾸미랑 삼겹살의 쭈삼 조합이...
9,휘경동 회기역 근처 맛집 고기가 푸짐한 회기왕갈비탕,2024. 5. 3. 21:44,추하!오늘은 휘경동 회기역 근처 맛집고기가 푸짐한 회기왕갈비탕포스팅을 하려고합니다....


In [83]:
file_name = f"{query}_{datetime.now().strftime('%Y%m%d')}.xlsx"
df.to_excel(file_name, index=False)

In [85]:
df['content'][0]

'안녕하세요이나입니다:)오늘은 혼밥 하기 좋은 회기역 맛집인 회기왕갈비탕에 방문한 후기를 들고 왔어요회기역 근처에서 촬영을 하고 혼밥을 해야 해서 뭘 먹지 고민하다가 혼밥 하기 좋다는 이야길 듣고 바로 후다닥 달려갔습니다 ㅎㅎ회기역에서 도보 5분 정도 거리로 굉장히 접근성도 좋아요ㅎㅎ그리고 외대 경희대 학생들도 가까워서 방문하기 좋을 것 같더라고요회기역 맛집은 석박지를 무려 직접 만든다고 해요힘드시겠다.. 그렇지만 소비자는 너무 맛있어요 츄릅김치 국내산? 보장된 거잖아요 히히회기왕갈비탕은 외부에서 봤을 때도 넓어 보였는데 들어가서 봐도 진짜 넓어요 굿굿쾌적하고 좌석도 많아서 단체로 와서 먹기에도 괜찮겠더라고요무려 1인 테이블도 있다는 점!! 혼밥 하기 좋은 회기역 맛집답게 1인 테이블까지~!~!근데 저는 혼밥 만렙이라서 4인 테이블 앉았어요점심시간 지나고 가서 자리가 많았기 때문이죠!!소갈비찜, 갈비탕 아롱사태 갈비탕 이렇게 메뉴가 깔끔하게 구성되어 있어요원래 맛집은 메뉴가 많이 없는 거 아시죠???기본 반찬 구성은 석박지, 쌈장, 고추, 마늘 이렇게 제공돼요!!기본 반찬 외에 필요한 건 셀프 코너에서!!셀프 코너에는 배추김치도 있어요근데 지금 포스팅하면서 알았는데 보리강정 먹어도 되는 거였냐고요진짜 몰랐네 알았으면 먹었는데…이거 진짜 좋아하는데속상하다…다음에는 꼭 먹어주겠다 보리강정 ㅡㅡ보리강정을 1봉지 3,000원2봉지 5,000원에 판매하고갈비탕 육수까지 같이 판매하고 있어요진짜 회기왕갈비탕 먹어보면 육수 따로 살만해요진짜 맛있거든요 ㅎㅎ혼밥 하기 좋은 회기역 맛집인 회기왕갈비탕은 무려 밥도 무한리필!!제가 아는 지인은 국밥 한 그릇에 세 공기, 다섯 공기 이렇게 먹는데 추천해 주고 싶네요 ㅋㅋㅋㅋㅋㅋ미숫가루도 후식으로 먹을 수 있게 준비되어 있어요열었는데 알아서 막 움직여서 놀랐어요 ㅋㅋㅋㅋㅋㅋㅋ진짜 당황 ㅇㅅㅇ혼밥 하기 좋은 회기역 맛집은 물도 맛집입니다무려 헛개열매차저 같은 술쟁이한테는 이렇게 식당에서 강제로 헛개열매차를 먹이면 오예입니다 ㅎㅎ제가 